In [1]:
# Spatial Extent of Chlorophyll-a Concentration 
# Estimates Using Developed Machine Learning Techniques - Objective 2
import rasterio
import numpy as np
import folium
# step 1 is to prepare the raster data!
# must include georeferencing information (like spatial reference system,
# coordinates, etc.)
with rasterio.open("/Users/erinfoley/Desktop/nasa2024/data/S2_SenecaLake_v2.tif") as src:
    raster_data = src.read()
    transform = src.transform # affine transform for pixel to geographic coords
    crs = src.crs # coordinate reference system

    # at this point, raster_data will contain pixel values that represent
    # chlorophyll-a values? idek
    # shape = (5, 10980, 10980)

# might need to reshape depending on the model we end up using. 
# for example, to reshape the data to (num pixels, num bands):
# num_pixels = raster_data.shape[1] * raster_data.shape[2]
# raster_flattened = raster_data.reshape(5, num_pixels).T

(5, 10980, 10980)


In [2]:
# step 2 is to apply the machine learning model
from joblib import load

# PUT MODEL HERE WHEN DONE
model = load('model')

# prediction for each pixel:
chlorophyll_a_predictions = model.predict(raster_flattened)
chlorophyll_a_predictions = chlorophyll_a_predictions.reshape(raster_data.shape[1], raster_data.shape[2])

In [ ]:
# step 3 is to make a heat map!
import matplotlib.pyplot as plt

plt.imshow(chlorophyll_a_predictions, cmap='viridis')
plt.colorbar(label='Chlorophyll-a Concentration (mg/m^3)')
plt.title('Chlorophyll-a Concentration Heat Map')
plt.xlabel('Pixel X')
plt.ylabel('Pixel Y')
plt.show()

In [ ]:
# step 4 is to transform pixel coordinates to geographic coordinates
def pixel_to_geo_coords(transform, x, y):
    x_geo, y_geo = rasterio.transform.xy(transform,x,y)
    return x_geo, y_geo

# generate geo coordinates
latitudes = []
longitudes = []
chlorophyll_vals = []

for row in range(chlorophyll_a_predictions.shape[1]):
    for col in range(chlorophyll_a_predictions.shape[2]):
        x_geo, y_geo = pixel_to_geo_coords(transform, row, col)
        latitudes.append(y_geo)
        longitudes.append(x_geo)
        chlorophyll_vals.append(chlorophyll_a_predictions[row,col])

heat_data = [[lat,lon,val] for lat, lon, val in zip(latitudes, longitudes, chlorophyll_vals)]

In [ ]:
# step 5 is to overlay predictions on a map (w/ Folium!)

# center of seneca lake: (42.7274, 76.9113)
lat_center = 42.7274
lon_center = -76.9113

lake_map = folium.Map(location=[latitude_center, logitude_center], zoom_start=12)

from folium.plugins import HeatMap
max_value = np.max(chlorophyll_vals)
normalized_heat_data = [[lat, lon, value / max_value] for lat, lon, value in heat_data]
HeatMap(normalized_heat_data, radius=10,blur=15,max_zoom=13).add_to(lake_map)

lake_map.save('chlorophyll_a_heatmap.html')
lake_map